In [2]:
import pandas as pd

"""
* header = 0
파일의 첫 번째 줄에 열 이름이 있음
* delimiter = \t
필드가 탭으로 구분됨
* quoting = 3
쌍따옴표를 무시
"""

# label이 있는 데이터 (sentiment)
train = pd.read_csv('../data/imdb/labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)

# label이 없는 데이터
test = pd.read_csv('../data/imdb/testData.tsv', header=0, delimiter='\t', quoting=3)

In [8]:
# 25,000개의 데이터, 3개의 칼럼(id, sentiment, review)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [9]:
# 25,000개의 데이터, 2개의 칼럼(id, review)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
id        25000 non-null object
review    25000 non-null object
dtypes: object(2)
memory usage: 390.7+ KB
